#### Thank You very much for passing and check this out 
- This is my very first Convolutional Neural Network
- In here you will find some amazing charts such as ***Circular Bar Chart**
- Some technique that I learn - such as *** apply Batch Normalizarion before Passing into Fuction/transformation ***

### Interesting fact
- I am surprised that the model didnt improve after applied Image Augmentation *** Maybe I was not doing properly***  please let your comment / or any feedback below 
- This is a Very common architecture but it was very porwerfull, Eventually i will use transfer learning with Google models to compare or learn how to improve this model 


# Libraries and Fuctions

In [ ]:
import numpy as np 
import pandas as pd 
import re

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf 
from keras.utils import to_categorical
import cv2

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
##### Plotting
def Plotting_NeuralNet (data):
    fig, ax = plt.subplots(1,2 , figsize = (20,7))
    # summarize history for accuracy
    ax[0].plot(data.history['accuracy'])
    ax[0].plot(data.history['val_accuracy'])
    ax[0].set_title('model accuracy')
    ax[0].legend(['train', 'test'], loc='upper left')

    # summarize history for loss
    ax[1].plot(data.history['loss'], label =['loss'])
    ax[1].plot(data.history['val_loss'] ,label =['val_loss'])
    ax[1].set_title('model loss')
    ax[1].legend(['train', 'test'], loc='upper left')
    plt.show()
    
#### Visualization
def get_label_rotation(angle, offset):
    # Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle + offset)
    if angle <= np.pi:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"
    return rotation, alignment

def add_labels(angles, values, labels, offset, ax):
    
    # This is the space between the end of the bar and the label
    padding = 4
    
    # Iterate over angles, values, and labels, to add all of them.
    for angle, value, label, in zip(angles, values, labels):
        angle = angle
        
        # Obtain text rotation and alignment
        rotation, alignment = get_label_rotation(angle, offset)

        # And finally add the text
        ax.text(x=angle, y=value + padding, 
            s=label, ha=alignment, va="center", 
            rotation=rotation, rotation_mode="anchor")

# Loading Data

In [ ]:
# Getting Data <----------
path = '../input/tmnist-alphabet-94-characters/94_character_TMNIST.csv'
df = pd.read_csv(path)
df.sample()

## Visualization

- The following Visualization is known as ***Circular bar chart*** each group A,B,C,D represent the different types of data, ***Numbers, Symbols, Letters, etc***
- The ***Left circular*** bar chart represents the number of pixels in each image - the represeantion might not be that accuracte because I had to scale and the visualization is for practicing purpose only.
- The  ***Rigth circular*** bar chart represents the number of imaes or distribution.
- ***Observation*** The data is well Distributed, each class has equal number of images as you can see in the Circular chart bar 

In [ ]:
# All labels
all_ = list(df['labels'].unique())

# Regex Pattern
pattern_uc = re.compile(r"[A-Z]")
pattern_lc = re.compile(r"[a-z]")
pattern_numbers = re.compile(r"[0-9]")
pattern_symbols = re.compile(r"[\W]|[\_\,]")

# Extracting Pattern
lower_case = pattern_lc.findall(str(all_))
Upper_case = pattern_uc.findall(str(all_))
Numbers_ = pattern_numbers.findall(str(all_))
Symbols_ = list(set(pattern_symbols.findall(str(all_))))
Symbols_.pop(27)

# Creating Gropus
group = 1
for list_ in (lower_case,Upper_case,Numbers_,Symbols_):
    df.loc[df['labels'].isin(list_), 'group'] = str(group)
    group += 1


In [ ]:
VALUES = df.groupby(['labels']).sum().reset_index()
VALUES = (VALUES.iloc[:,1:-1].sum(axis=1))*0.000001
IMAGES_ = (df.groupby(['labels']).count()['names'].values)*0.025
LABELS = df.groupby(['labels','group']).count().reset_index().sort_values('group')['labels'].values
GROUP = df.groupby(['labels','group']).count().reset_index().sort_values('group')['group'].values
GROUPS_SIZE = [26, 26, 10, 32]

In [ ]:
# Add three empty bars to the end of each group
PAD = 3
ANGLES_N = len(VALUES) + PAD * len(np.unique(GROUP))
ANGLES = np.linspace(0, 2 * np.pi, num=ANGLES_N, endpoint=False)
WIDTH = (2 * np.pi) / len(ANGLES)

offset = 0
OFFSET = np.pi / 2
IDXS = []
GROUPS_SIZE = [26, 26, 10, 32]

for size in GROUPS_SIZE:
    IDXS += list(range(offset + PAD, offset + size + PAD))
    offset += size + PAD

fig, ax = plt.subplots(1,2,figsize=(20, 10), subplot_kw={"projection": "polar"})
ax[0].set_theta_offset(OFFSET)
ax[0].set_ylim(-100, 100)
ax[0].set_frame_on(False)
ax[0].xaxis.grid(False)
ax[0].yaxis.grid(False)
ax[0].set_xticks([])
ax[0].set_yticks([])

COLORS = [f"C{i}" for i, size in enumerate(GROUPS_SIZE) for _ in range(size)]

ax[0].bar(
    ANGLES[IDXS], VALUES, width=WIDTH, color=COLORS, 
    edgecolor="white", linewidth=2
)

add_labels(ANGLES[IDXS], VALUES, LABELS, OFFSET, ax[0])
offset = 0 
for group, size in zip(["Lower","Upper","Numb","Symb"], GROUPS_SIZE):
    # Add line below bars
    x1 = np.linspace(ANGLES[offset + PAD], ANGLES[offset + size + PAD - 1], num=50)
    ax[0].plot(x1, [-5] * 50, color="#333333")
    
    # Add text to indicate group
    ax[0].text(
        np.mean(x1), -20, group, color="#333333", fontsize=14, 
        fontweight="bold", ha="center", va="center"
    )
    
    # Add reference lines at 20, 40, 60, and 80
    x2 = np.linspace(ANGLES[offset], ANGLES[offset + PAD - 1], num=50)
    ax[0].plot(x2, [20] * 50, color="#bebebe", lw=0.8)
    ax[0].plot(x2, [40] * 50, color="#bebebe", lw=0.8)
    ax[0].plot(x2, [60] * 50, color="#bebebe", lw=0.8)
    ax[0].plot(x2, [80] * 50, color="#bebebe", lw=0.8)
    
    offset += size + PAD
    
# -------------------------------------
PAD = 3
ANGLES_N = len(IMAGES_) + PAD * len(np.unique(GROUP))
ANGLES = np.linspace(0, 2 * np.pi, num=ANGLES_N, endpoint=False)
WIDTH = (2 * np.pi) / len(ANGLES)

offset = 0
OFFSET = np.pi / 2
IDXS = []
GROUPS_SIZE = [26, 26, 10, 32]

for size in GROUPS_SIZE:
    IDXS += list(range(offset + PAD, offset + size + PAD))
    offset += size + PAD
    
ax[1].set_theta_offset(OFFSET)
ax[1].set_ylim(-100, 100)
ax[1].set_frame_on(False)
ax[1].xaxis.grid(False)
ax[1].yaxis.grid(False)
ax[1].set_xticks([])
ax[1].set_yticks([])

COLORS = [f"C{i}" for i, size in enumerate(GROUPS_SIZE) for _ in range(size)]

ax[1].bar(
    ANGLES[IDXS], IMAGES_, width=WIDTH, color=COLORS, 
    edgecolor="white", linewidth=2
)

add_labels(ANGLES[IDXS], IMAGES_, LABELS, OFFSET, ax[1])
offset = 0 
for group, size in zip(["Lower","Upper","Numb","Symb"], GROUPS_SIZE):
    # Add line below bars
    x1 = np.linspace(ANGLES[offset + PAD], ANGLES[offset + size + PAD - 1], num=50)
    ax[1].plot(x1, [-5] * 50, color="#333333")
    
    # Add text to indicate group
    ax[1].text(
        np.mean(x1), -20, group, color="#333333", fontsize=14, 
        fontweight="bold", ha="center", va="center"
    )
    
    # Add reference lines at 20, 40, 60, and 80
    x2 = np.linspace(ANGLES[offset], ANGLES[offset + PAD - 1], num=50)
    ax[1].plot(x2, [20] * 50, color="#bebebe", lw=0.8)
    ax[1].plot(x2, [40] * 50, color="#bebebe", lw=0.8)
    ax[1].plot(x2, [60] * 50, color="#bebebe", lw=0.8)
    ax[1].plot(x2, [80] * 50, color="#bebebe", lw=0.8)
    
    offset += size + PAD
 

# Spliting Labels & Images 

In [ ]:
X = df.iloc[:, 2:-1].astype('float32') # Splitting and turning into Float
y  = df[['labels']] #Extracting Labels

In [ ]:
labels = y['labels'].unique()
values = [num for num in range(len(df['labels'].unique()))]
label_dict= dict(zip(labels,values)) #Creating Dictionary 
label_dict_inv = dict(zip(values,labels))

print(sorted(label_dict.items(), key=lambda x:x[1])[35:45])  #For visualization Purpose

In [ ]:
# Transforming
y['labels'].replace(label_dict, inplace=True) #Maping Values
y.tail(5)

# Creating Training and Test Sets

### Reshaping and Displaying some Images

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
Length, Height = 28,28  # <---- Defining LxH 
NCl = y_train.nunique()[0] # Unique targets -- > 94 

# ------>  N of images 28x28
X_train = np.reshape(X_train.values, (X_train.shape[0] ,Length, Height)) 
X_test = np.reshape(X_test.values, (X_test.shape[0] ,Length, Height))

# -------> Target into Categorical Values
y_train = to_categorical(y_train, NCl, dtype='int' )
y_test = to_categorical(y_test, NCl, dtype='int' )

print(f'X:Train, Test data shape:{X_train.shape},{X_test.shape}')
print(f'Y:Train, Test data shape:{y_train.shape},{y_test.shape}')

In [ ]:
random = shuffle(X_train[:500]) #Randomly shuffle (array  in a consistent way)
fig,ax = plt.subplots(3,4 , figsize = (10,10)) 
axes = ax.flatten()

for i in range(12):
    _,shu = cv2.threshold(random[i], 30, 200, cv2.THRESH_BINARY)
    axes[i].imshow(np.reshape(random[i], (Length, Height)), cmap = 'Greys')
plt.show()

# Building Deep Learning FrameWork

- This is a representation of the Architecture that Iw as using 
Started with 32 Conv - > Apply some Batch Normalization -> ReLu Transformation X 3 - > Flatten - Dense Layer 
*** My apology*** , the visualization wasnt that good, in my defense I can say I was in my way to my job when I posted this "Code".

In [ ]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D,Flatten,Dropout,BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping

In [ ]:
from IPython.display import display
from PIL import Image
path=('../input/convo-image/Convo.png')
display(Image.open(path))

### Reshaping for CNN

In [ ]:
RGB = 1  # In this case only one instead of 3 because we dont have Color images
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],X_train.shape[2], RGB)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],RGB)


X_train = X_train/255
X_test = X_test/255
print(f'Train, Test shapes: {X_train.shape},{X_test.shape}')

## First Convolutional Architecture

In [ ]:
model = Sequential ()

# Conv -> Maxpool - Dropout [1st - 4rd] ~ Flatten - >  Dense - Dense - output 
model.add(Conv2D(filters = 32 , kernel_size = (3,3),input_shape = (Length, Height, RGB), padding = 'same',))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32 , kernel_size = (3,3) ,padding = 'same'))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64 , kernel_size = (3,3) ,padding = 'same'))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(350))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(NCl, activation = 'softmax'))
# model.summary()

In [ ]:
optimizer  = Adam(learning_rate=0.01)
callback =EarlyStopping(monitor='loss', patience=5)
Batch_ = 64
Epochs_ = 50

#Training -------------- >
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
history= model.fit(X_train,y_train, validation_data = (X_test,y_test),batch_size = Batch_ ,
                   epochs = Epochs_, verbose = 0)

score = model.evaluate(X_test,y_test, batch_size = Batch_)
print(f"Loss:{round(score[0],4)}")
print(f"Test Accuracy:{round(score[1],4)}")
Plotting_NeuralNet(history)

## Making Prediction

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(8,9))
axes = axes.flatten()

for i,ax in enumerate(axes):
    img = np.reshape(X_test[i], (28,28)) # reshaping it for displaying
    ax.imshow(img, cmap="Greys")
    img_final =np.reshape(img, (1,28,28,1)) # reshapng it for passing into model for prediction
    pred = label_dict_inv[np.argmax(model.predict(img_final))]
    ax.set_title("Prediction: "+pred)
    ax.grid()

## Second Convolutional Architecture + Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1)

In [ ]:
Batch_ = 62
history= model.fit_generator(datagen.flow(X_train,y_train,batch_size = Batch_), 
                             validation_data = (X_test,y_test), epochs = Epochs_,callbacks=[callback], verbose = 0)
score = model.evaluate(X_test,y_test, batch_size = Batch_)
print(f"Test Score:{round(score[0],4)}")
print(f"Test Accuracy:{round(score[1],4)}")
Plotting_NeuralNet(history)

### Making Predictions

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(8,9))
axes = axes.flatten()

for i,ax in enumerate(axes):
    img = np.reshape(X_test[i], (28,28)) # reshaping it for displaying
    ax.imshow(img, cmap="Greys")
    img_final =np.reshape(img, (1,28,28,1)) # reshapng it for passing into model for prediction
    pred = label_dict_inv[np.argmax(model.predict(img_final))]
    ax.set_title("Prediction: "+pred)
    ax.grid()

# Transfer Learning and More......